## Machine Learning on Kaggle Socccer Dataset (clustering model)
## k-means clustering

In [1]:
import sqlite3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.preprocessing import scale
%matplotlib inline

Download: [Kaggle Socccer Data set](https://www.kaggle.com/hugomathien/soccer)

In [2]:
# Create connection and ingest data
cnx = sqlite3.connect('database/database.sqlite')
df = pd.read_sql_query("SELECT pp.player_name, atr.* FROM Player_Attributes atr inner join Player pp on pp.player_api_id = atr.player_api_id", cnx)

In [3]:
# Exploring Data
df.describe(include = "all").transpose()

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
player_name,183978,10848,Danilo,108,NaN,NaN,NaN,NaN,NaN,NaN,NaN
id,183978,NaN,NaN,NaN,91989.5,53110,1,45995.2,91989.5,137984,183978
player_fifa_api_id,183978,NaN,NaN,NaN,165672,53851.1,2,155798,183488,199848,234141
player_api_id,183978,NaN,NaN,NaN,135901,136928,2625,34763,77741,191080,750584
date,183978,197,2007-02-22 00:00:00,11794,NaN,NaN,NaN,NaN,NaN,NaN,NaN
overall_rating,183142,NaN,NaN,NaN,68.6,7.04114,33,64,69,73,94
potential,183142,NaN,NaN,NaN,73.4604,6.59227,39,69,74,78,97
preferred_foot,183142,2,right,138409,NaN,NaN,NaN,NaN,NaN,NaN,NaN
attacking_work_rate,180748,8,medium,125070,NaN,NaN,NaN,NaN,NaN,NaN,NaN
defensive_work_rate,183142,19,medium,130846,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
# Testing for repeated player_api_id
df.player_api_id.value_counts().head()

210278    56
41269     56
42116     55
26472     54
179795    53
Name: player_api_id, dtype: int64

In [5]:
# Explore for a method to exclude duplicates - column "Date"
df.date.value_counts().head()

2007-02-22 00:00:00    11794
2013-09-20 00:00:00     6543
2011-08-30 00:00:00     6525
2015-09-21 00:00:00     6522
2012-08-31 00:00:00     6495
Name: date, dtype: int64

In [6]:
df = df.loc[df.date=="2015-09-21 00:00:00"]

In [7]:
# Check for info still duplicated
df.player_api_id.value_counts().head()

37254     2
110189    2
163838    2
193866    2
346111    1
Name: player_api_id, dtype: int64

In [8]:
player_api_id_repeted = [37254, 110189, 163838, 193866]
df.loc[df.player_api_id.isin(player_api_id_repeted)]

,player_name,id,player_fifa_api_id,player_api_id,date,overall_rating,potential,preferred_foot,attacking_work_rate,defensive_work_rate,...,vision,penalties,marking,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes
7426,Alex,7427,198033,163838,2015-09-21 00:00:00,73.0,78.0,left,medium,medium,...,72.0,54.0,27.0,26.0,28.0,16.0,7.0,12.0,8.0,15.0
7452,Alex,7453,198394,163838,2015-09-21 00:00:00,NaN,NaN,None,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
107760,Mahamadou Samassa,107761,177485,37254,2015-09-21 00:00:00,72.0,75.0,right,medium,medium,...,12.0,18.0,12.0,11.0,12.0,76.0,71.0,64.0,66.0,79.0
107787,Mahamadou Samassa,107788,190195,37254,2015-09-21 00:00:00,NaN,NaN,None,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
129776,Nacer Chadli,129777,148581,110189,2015-09-21 00:00:00,79.0,82.0,right,medium,medium,...,78.0,76.0,37.0,48.0,44.0,12.0,11.0,12.0,10.0,13.0
129801,Nacer Chadli,129802,200529,110189,2015-09-21 00:00:00,NaN,NaN,None,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
138959,Papy Djilobodji,138960,148337,193866,2015-09-21 00:00:00,78.0,79.0,left,None,8,...,50.0,27.0,75.0,82.0,89.0,12.0,13.0,9.0,6.0,10.0
138983,Papy Djilobodji,138984,197937,193866,2015-09-21 00:00:00,NaN,NaN,None,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
# Check for NaN at "Overall rating" column
df.overall_rating.isnull().sum()

# Obs.: It is just these four repeated players with NaN values

4

In [10]:
# Data Cleaning Steps
before_rows = df.shape[0]
df = df.dropna()
after_rows = df.shape[0]
print("It was deleted %d rows" %(before_rows - after_rows))

It was deleted 4 rows


In [11]:
# Exploring final dataset
df.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
id,6518.0,92627.434489,53436.167338,3.0,46148.75,93393.5,139015.25,183910.0
player_fifa_api_id,6518.0,182440.111077,42248.975540,2.0,171885.00,191687.5,207922.25,231137.0
player_api_id,6518.0,186725.952900,166841.362006,2752.0,40445.75,150494.5,276737.00,744907.0
overall_rating,6518.0,69.975299,6.186119,48.0,66.00,70.0,74.00,94.0
potential,6518.0,73.448911,6.051496,51.0,69.00,73.0,77.00,95.0
crossing,6518.0,55.005830,18.510649,7.0,45.00,60.0,68.00,93.0
finishing,6518.0,49.069040,19.824003,6.0,33.00,53.0,65.00,95.0
heading_accuracy,6518.0,57.110003,17.568429,8.0,50.00,60.0,69.00,95.0
short_passing,6518.0,63.347346,14.151592,11.0,59.00,66.0,73.00,93.0
volleys,6518.0,48.255907,18.697495,5.0,34.00,51.0,63.00,92.0


In [12]:
# Use this command to bring information regarding a player
df.loc[df.player_name=="Cristiano Ronaldo"].transpose()

,33332
player_name,Cristiano Ronaldo
id,33333
player_fifa_api_id,20801
player_api_id,30893
date,2015-09-21 00:00:00
overall_rating,93
potential,93
preferred_foot,right
attacking_work_rate,high
defensive_work_rate,low


In [13]:
# correlation between potencial and overall_rating
df['overall_rating'].corr(df.potential)

0.7969820121911912

In [14]:
# Assign features to a list
cols = ['potential',  'crossing', 'finishing', 'heading_accuracy',
       'short_passing', 'volleys', 'dribbling', 'curve', 'free_kick_accuracy',
       'long_passing', 'ball_control', 'acceleration', 'sprint_speed',
       'agility', 'reactions', 'balance', 'shot_power', 'jumping', 'stamina',
       'strength', 'long_shots', 'aggression', 'interceptions', 'positioning',
       'vision', 'penalties', 'marking', 'standing_tackle', 'sliding_tackle',
       'gk_diving', 'gk_handling', 'gk_kicking', 'gk_positioning',
       'gk_reflexes']

In [15]:
# check how the features are correlated with the overall rating

correlation = []
for f in cols:
    related = df['overall_rating'].corr(df[f])
    correlation.append(related)
    #print("%s: %f" % (f,related))

In [16]:
df_corr = pd.DataFrame(
    {'feature': cols,
     'correlation': correlation
    }, columns = [ 'feature', 'correlation' ])


In [17]:
df_corr.sort_values(by=['correlation'], ascending=False)

,feature,correlation
14,reactions,0.818948
0,potential,0.796982
4,short_passing,0.400583
9,long_passing,0.391820
24,vision,0.390555
10,ball_control,0.357703
16,shot_power,0.334529
7,curve,0.318705
20,long_shots,0.314656
5,volleys,0.295660


### Perform k-means clustering

In [18]:
# Select Features
select5features = ['reactions', 'crossing', 'marking', 'interceptions', 'gk_handling']

df_select = df[select5features].copy(deep=True)
# In this case, we do not need to scale features
#data = scale(df_select)

In [19]:
# Define number of clusters
noOfClusters = 4

# Train a model
model = KMeans(init='k-means++', n_clusters=noOfClusters, n_init=20).fit(df_select)



#Kmean.cluster_centers_
centers = model.cluster_centers_

In [20]:
centers.shape

(4, 5)

In [21]:
centers

array([[66.8614082 , 60.0543672 , 26.78386809, 33.01336898, 10.76203209],
       [66.60172414, 14.11896552, 13.67931034, 20.28103448, 67.41551724],
       [69.72525168, 67.49790268, 65.46392617, 68.1136745 , 10.66317114],
       [62.75114504, 41.72671756, 67.07251908, 66.67251908, 10.60763359]])

In [22]:
# Understanding the clusters
df_cluster = pd.DataFrame(centers, columns= select5features)
df_cluster.index.name = 'cluster'
df_cluster

,reactions,crossing,marking,interceptions,gk_handling
cluster,,,,,
0,66.861408,60.054367,26.783868,33.013369,10.762032
1,66.601724,14.118966,13.679310,20.281034,67.415517
2,69.725252,67.497903,65.463926,68.113674,10.663171
3,62.751145,41.726718,67.072519,66.672519,10.607634


In [23]:
# Number of players in each cluster
pd.value_counts(model.labels_, sort=False)

0    2244
1     580
2    2384
3    1310
dtype: int64

In [24]:
model.labels_

array([3, 2, 0, ..., 0, 0, 3])

In [25]:
# Including Prediction column to the initial Dataframe
df['prediction']= model.labels_

In [26]:
df.head()

,player_name,id,player_fifa_api_id,player_api_id,date,overall_rating,potential,preferred_foot,attacking_work_rate,defensive_work_rate,...,penalties,marking,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes,prediction
2,Aaron Appindangoye,3,218353,505942,2015-09-21 00:00:00,62.0,66.0,right,medium,medium,...,48.0,65.0,66.0,69.0,6.0,11.0,10.0,8.0,8.0,3
12,Aaron Cresswell,13,189615,155782,2015-09-21 00:00:00,73.0,77.0,left,medium,medium,...,59.0,76.0,75.0,78.0,14.0,7.0,9.0,9.0,12.0,2
40,Aaron Doran,41,186170,162549,2015-09-21 00:00:00,66.0,70.0,right,medium,medium,...,61.0,23.0,22.0,24.0,16.0,11.0,12.0,9.0,13.0,0
67,Aaron Galindo,68,140161,30572,2015-09-21 00:00:00,69.0,69.0,right,medium,medium,...,37.0,72.0,71.0,68.0,15.0,12.0,13.0,12.0,11.0,3
88,Aaron Hughes,89,17725,23780,2015-09-21 00:00:00,70.0,70.0,right,medium,medium,...,45.0,75.0,73.0,71.0,8.0,6.0,16.0,12.0,11.0,3


In [27]:
# Show features and predicted cluster for some players
player_names = ["Cristiano Ronaldo", "Sergio Ramos", "Gianluigi Buffon", "Miranda"]

df.loc[df.player_name.isin(player_names)].transpose()

,33332,63847,127009,161329
player_name,Cristiano Ronaldo,Gianluigi Buffon,Miranda,Sergio Ramos
id,33333,63848,127010,161330
player_fifa_api_id,20801,1179,168609,155862
player_api_id,30893,30717,19327,30962
date,2015-09-21 00:00:00,2015-09-21 00:00:00,2015-09-21 00:00:00,2015-09-21 00:00:00
overall_rating,93,84,84,87
potential,93,84,84,87
preferred_foot,right,right,right,right
attacking_work_rate,high,medium,medium,high
defensive_work_rate,low,medium,medium,medium


In [28]:
# Use this command to select the top 5 players sorted by overall_rating from a specific cluster
prediction_cluster = 0
df.loc[df.prediction==prediction_cluster].sort_values(by=['overall_rating'], ascending=False).head().transpose()

,102484,33332,16462,105984,183672
player_name,Lionel Messi,Cristiano Ronaldo,Arjen Robben,Luis Suarez,Zlatan Ibrahimovic
id,102485,33333,16463,105985,183673
player_fifa_api_id,158023,20801,9014,176580,41236
player_api_id,30981,30893,30834,40636,35724
date,2015-09-21 00:00:00,2015-09-21 00:00:00,2015-09-21 00:00:00,2015-09-21 00:00:00,2015-09-21 00:00:00
overall_rating,94,93,90,90,89
potential,95,93,90,90,89
preferred_foot,left,right,left,right,right
attacking_work_rate,medium,high,high,high,medium
defensive_work_rate,low,low,low,medium,low
